# CFFI + Numba basics

In [9]:
import cffi
import numba
import numpy as np
from numba.core.typing import cffi_utils

In [2]:
# lib_dir = os.path.expanduser('/mnt/work_folder/convex-plasticity/optimisation/mylib/')
lib_dir = os.path.join(os.getcwd(), 'lib/')
include_dir = os.path.join(os.getcwd(), 'include/')

ffibuilder = cffi.FFI()
ffibuilder.set_source('_mylib',
  r'#include <mylib.h>',
  include_dirs = [include_dir],
  libraries = ['mylib'],
  library_dirs = [lib_dir],
)

ffibuilder.cdef("""
  double sum(double a, double b);

  typedef struct {
    double a;
    int b;
    double c[3];
} mystruct_t;

""")

ffibuilder.compile()

'/mnt/work_folder/convex-plasticity/demo/cffi_basics/_mylib.cpython-39-x86_64-linux-gnu.so'

In [3]:
import _mylib

cffi_utils.register_module(_mylib)

mystruct_numba_t = cffi_utils.map_type(_mylib.ffi.typeof('mystruct_t'), use_record_dtype=True)
cffi_utils.register_type(_mylib.ffi.typeof('mystruct_t'), mystruct_numba_t)

In [5]:
@numba.njit()
def func(a, b):
    return _mylib.lib.sum(a, b)
func(1., 1.)

2.0

In [12]:
mystruct = _mylib.ffi.new("mystruct_t *", {})
mystruct_numpy = np.ndarray(buffer=_mylib.ffi.buffer(mystruct), dtype=numba.np.numpy_support.as_dtype(mystruct_numba_t), shape=1,)

mystruct_address = int(_mylib.ffi.cast('size_t', mystruct))

print(mystruct_address, mystruct_numpy.ctypes.data)

51180416 51180416
